**Import dependecies**

In [1]:
'''
%conda install -c apple tensorflow-deps scipy #macos
%pip install tensorflow-macos tensorflow-metal #macos

%pip install --upgrade tensorflow 
%pip install pydot graphviz

%pip install pandas numpy pillow matplotlib scipy autopep8 pydot
'''

'\n%conda install -c apple tensorflow-deps scipy #macos\n%pip install tensorflow-macos tensorflow-metal #macos\n\n%pip install --upgrade tensorflow \n%pip install pydot graphviz\n\n%pip install pandas numpy pillow matplotlib scipy autopep8 pydot\n'

In [2]:
import tensorflow as tf
%load_ext tensorboard

In [3]:
#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
import os
from tensorflow import keras
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import Model
import datetime

fruit = "apple"
model_current = "vgg16"
curr_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
run_prefix = f'./runs/{fruit}/{model_current}/{curr_datetime}/'
#results_prefix = f'./results/{fruit}/{model_current}/{curr_datetime}/'
log_dir = f'logs/fit/{fruit}/{model_current}/{curr_datetime}'
os.makedirs(run_prefix, exist_ok=True)
#os.makedirs(results_prefix, exist_ok=True)
run_prefix


'./runs/apple/vgg16/20220802-152414/'

**Create an imagedatagenerator object to label data**

In [6]:
input_size = 224
batch_size = 16
train_size = 8000
test_size = 2000
learning_rate = 0.001
epochs = 20
dropout = 0.0
dense_size = 1024

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
train_dir = f"./dataset_frutas_colab/{fruit}/train"
test_dir = f"./dataset_frutas_colab/{fruit}/test"
train_generator = train_datagen.flow_from_directory(train_dir,batch_size=batch_size,class_mode = 'binary', shuffle= True, target_size=(input_size,input_size))
test_generator = test_datagen.flow_from_directory(test_dir,batch_size=batch_size,class_mode = 'binary',shuffle= True, target_size=(input_size,input_size))

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


VGG16 model

In [7]:
pre_trained_model = VGG16(
    input_shape = (input_size,input_size,3),
    include_top = False,
    weights='imagenet'
)


Start transfer learning process

In [8]:
for layer in pre_trained_model.layers:
  layer.trainable = False

In [9]:
#Flatten
x = layers.Flatten()(pre_trained_model.output)
#Fully connected layer con 1,024 hidden units y ReLU
x = layers.Dense(dense_size, activation='relu')(x)
x = layers.Dense(dense_size/2, activation='relu')(x)
#Dropout rate
if (dropout>0):
    x = layers.Dropout(dropout)(x)
#Binary classification and sigmoid func
x = layers.Dense(1,activation='sigmoid')(x)

model = Model(pre_trained_model.input,x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics = [tf.keras.metrics.Accuracy(),
                         tf.keras.metrics.Precision(), 
                         tf.keras.metrics.Recall()])

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
tf.keras.utils.plot_model(
    model,
    to_file=f"{run_prefix}model.png",
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True,
)
print("Saved Model Image")

Saved Model Image


In [12]:
import datetime

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(
    train_generator,
    validation_data = test_generator,
    steps_per_epoch = train_size/batch_size,
    epochs = epochs,
    validation_steps = test_size/batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/20
500/500 [==============================] - 174s 336ms/step - loss: 0.5684 - accuracy: 0.0072 - precision: 0.8610 - recall: 0.8637 - val_loss: 0.0826 - val_accuracy: 0.0060 - val_precision: 0.9845 - val_recall: 0.9550
Epoch 2/20
500/500 [==============================] - 168s 336ms/step - loss: 0.1364 - accuracy: 0.0878 - precision: 0.9537 - recall: 0.9572 - val_loss: 0.1966 - val_accuracy: 0.1700 - val_precision: 0.9120 - val_recall: 0.9950
Epoch 3/20
500/500 [==============================] - 167s 334ms/step - loss: 0.1181 - accuracy: 0.1631 - precision: 0.9657 - recall: 0.9712 - val_loss: 0.1039 - val_accuracy: 0.2025 - val_precision: 0.9433 - val_recall: 0.9990
Epoch 4/20
500/500 [==============================] - 167s 334ms/step - loss: 0.0852 - accuracy: 0.2386 - precision: 0.9750 - recall: 0.9770 - val_loss: 0.0607 - val_accuracy: 0.1925 - val_precision: 0.9809 - val_recall: 0.9740
Epoch 5/20
500/500 [==============================] - 164s 329ms/step - loss: 0.0650 - a

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
precision = history.history['precision']
recall = history.history['recall']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.savefig(f"{run_prefix}acc.svg")
plt.figure()


plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig(f"{run_prefix}loss.svg")

plt.show()

In [ ]:
#model.save(f"{run_prefix}{model_current}.h5")

In [ ]:
import csv

header = ["train_acc", "val_acc", "train_loss", "val_loss", "precision", "recall"]

with open(f'{run_prefix}{epochs}-{dropout}-{batch_size}-{learning_rate}-{input_size}-RMS-metrics.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)
    for idx in range(len(acc)):
        line = [acc[idx],val_acc[idx],loss[idx],val_loss[idx], precision[idx], recall[idx]]        
        writer.writerow(line)


Probar la prediccion

In [ ]:
'''
paths = ["./fresh1.jpg", "./fresh2.jpg", "./fresh3.jpg", "./fresh4.png", "./rotten1.jpg", "./rotten2.png", "./rotten3.png", "./rotten4.png"]

model = keras.models.load_model('16_20_0.0_inceptionv3.h5')

for path in paths:
    img = image.load_img(path, target_size=(250, 250))

    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tensorflow.expand_dims(img_array, 0)

    #predict
    preds = model.predict(img_array)
    print(preds)
'''